Se dispone de varios archivos raster en formato ".tif", donde cada uno almacena el pronóstico de precipitación para el día siguiente en todo el dominio geográfico de Guatemala; estas predicciones fueron obtenidas a partir del modelo WRF.

Para cada uno de los archivos ".tif" se realiza lo siguiente:

01. Se toma el valor asociado a la ubicación geográfica de cada una de las estaciones meteorológicas.
02. Para cada estación meteorológica, se generan las variables objetivo de estudio:
    - Anomalía de precipitación del día siguiente con respecto al promedio de la Climatología
    - Clasificación binaria de la precipitación del día siguiente como =0 ó >0
    - Clasificación de la precipitación del día siguiente en cotas percentiles
03. Luego, se agrega a un dataframe el nombre de la estación, la fecha, la latitud y longitud de la estación, el número de mes (de 1 a 12), así como las tres variables objetivo pronosticadas por WRF

Al finalizar el proceso, el dataframe es exportado a un archivo en formato ".xlsx"

# IMPORTACIÓN DE DATOS Y LIBRERÍAS

In [ ]:
import pandas as pd
import rasterio
from rasterio.sample import sample_gen
import numpy as np
import os
from datetime import datetime, timedelta

In [ ]:
# Cargar datos desde Excel
estaciones_df = pd.read_excel('Datos_estaciones.xlsx')  # Archivo con latitud y longitud de cada estación
climatologia_df = pd.read_excel('Percentiles.xlsx')  # Archivo con climatología (media y percentiles para cada estación en cada día del ano)

# DEFINICIÓN DE FUNCIONES

In [ ]:
# Función para extraer la precipitación de un archivo .tiff en todas las estaciones

def obtener_precipitacion_en_estaciones(ruta_tiff, latitudes, longitudes):
    coords = [(lon, lat) for lon, lat in zip(longitudes, latitudes)]
    with rasterio.open(ruta_tiff) as src:
        precipitaciones = list(src.sample(coords))
        precipitaciones = [x[0] for x in precipitaciones]  # Obtenemos solo la precipitación
    return precipitaciones

In [ ]:
# Función para calcular anomalía de precipitación

def anom_prec(prec, media):
    anom = prec - media
    suma = anom + media
    if suma < 0:
        anom = -media
    return anom

In [ ]:
# Función de clasificación binaria de la lluvia como =0 ó >0

def Lluvia_binaria(prec):
    if prec == 0:
        return 0
    else:
        return 1

In [ ]:
# Función para clasificar por percentiles

def clasificar_percentiles(prec, p1_3, p2_3):
    if p1_3 == 0 and p2_3 == 0 and prec == 0:
        return 0
    else:
        if prec <= p1_3:
            return -1
        elif prec >= p2_3:
            return 1
        else:
            return 0

# PROCESAMIENTO DE ARCHIVOS .TIFF

In [ ]:
# Procesar todos los archivos .tiff
resultados = []
ruta_tiffs = 'Archivos_tiff/'  # Ruta donde están almacenados los archivos .tiff

In [ ]:
# Ciclo principal de ejecución:

for archivo_tiff in os.listdir(ruta_tiffs):
    if archivo_tiff.endswith('.tif'):
        # Extraer la fecha del nombre del archivo (asumiendo formato [yyyy][mm][dd].tiff)
        fecha_str = archivo_tiff.split('.')[0]
        fecha = datetime.strptime(fecha_str, '%Y%m%d')
        mes = fecha.month
        dia = fecha.day
        dia_del_año = f'{mes}-{dia}'
        
        if dia<10:
            dia_del_año = f'{mes}-0{dia}'  #Agregar un cero para que la fecha sea 11-01 y no 11-1, por ejemplo
            
        if mes<10:
            dia_del_año = f'0{mes}-{dia}'  #Agregar un cero para que la fecha sea 01-11 y no 1-11, por ejemplo
        
        if dia<10 and mes<10:
            dia_del_año = f'0{mes}-0{dia}'  #Agregar un cero para que la fecha sea 01-01 y no 1-1, por ejemplo
        
        # Extraer precipitación en las coordenadas de cada estación
        latitudes = estaciones_df['Latitud'].values
        longitudes = estaciones_df['Longitud'].values
        precipitaciones = obtener_precipitacion_en_estaciones(os.path.join(ruta_tiffs, archivo_tiff), latitudes, longitudes)
        
        for i, estacion in estaciones_df.iterrows():
            nombre_estacion = estacion['ESTACIÓN']
            latitud = estacion['Latitud']
            longitud = estacion['Longitud']
            
            # Obtener los datos de climatología para la fecha y estación
            clima = climatologia_df[(climatologia_df['ESTACIÓN'] == nombre_estacion) & (climatologia_df['DiaDelAno'] == dia_del_año)]
            
            media_clim = clima['Media'].values[0]
            p1_3 = clima['Percentil_1_/3'].values[0]
            p2_3 = clima['Percentil_2/3'].values[0]
            
            # Precipitación predicha
            precip_predicha = precipitaciones[i]
            
            # Anomalía de precipitación
            anomalia_prec = anom_prec(precip_predicha, media_clim)
            
            # Clasificación de Lluvia binaria
            lluvia_binaria = Lluvia_binaria(precip_predicha)
            
            # Clasificación por percentiles
            percentil_clase = clasificar_percentiles(precip_predicha, p1_3, p2_3)
            
            # Guardar resultados
            resultados.append([nombre_estacion, fecha, latitud, longitud, mes, percentil_clase, lluvia_binaria, anomalia_prec])

# Crear DataFrame final

In [ ]:
# Crear DataFrame final
resultados_df = pd.DataFrame(resultados, columns=['Estacion', 'Fecha', 'Latitud', 'Longitud', 'Mes', 'percentil_WRF', 'Lluvia_Binaria_WRF', 'Anom_prec_mañana_WRF'])

# Exportar a Excel
resultados_df.to_excel('PronósticosWRF.xlsx', index=False)